# READ Nested Data from Twitter API: JSON to DF

# <font color=red>Mr Fugu Data Science</font>

# (◕‿◕✿)

# Purpose & Outcome:

Use the developer: `Twitter API` to extract nested tweet data and insert into a dataframe.
+ As a nested DF
+ Then as a parsed DF
+ Use `json_normalize()`
+ See `df.explode()`
+ Ultimately, manipulate JSON data

**You have the ability to obtain (*unnested*) tweets without going through this process. But, it is an exercise of practicing (*parsing nested*) json data.**


In [11]:
import pandas as pd
import collections as cc # for default dictionary
import base64 # encoding
import requests # make a connection
import re # regex
import json
from collections import defaultdict
# from flatten_json import flatten

+ You need to have a Twitter account setup prior to using the devoper api.
    + Once you have that; setup a deveoper account.
        + Then you will need to create an app. which allows you to use the API
       

In [ ]:
# https://developer.twitter.com/en/docs/basics/authentication/api-reference/token

# https://developer.twitter.com/en/docs/basics/authentication/guides/authentication-best-practices

In [12]:
# CREDENTIALS REPLACE WITH YOUR OWN:

client_key = '' # PUT YOURS HERE

client_secret = '' # PUT YOURS HERE

# I suggest using a config file to store your credentials!


In [13]:
# Call your two keys: and encode 
key_secret = '{}:{}'.format(client_key, client_secret).encode('ascii')

# Use base64 to encode the keys to binary
b64_encoded_key = base64.b64encode(key_secret)

# Convert to ascii
b64_encoded_key = b64_encoded_key.decode('ascii')

+ What is **OAuth**?
    + protocol giving an application permission to act on its behalf; without sharing its password. 


`auth_data`: There are several grant types
+ `Authorization`: used for running web servers, mobile or browser type apps
+ `Client Credentials`: can give access to an application without the user present
+ `Passwords`: logging into an account
  
https://developer.twitter.com/en/docs/basics/authentication/overview

In [14]:
base_url = 'https://api.twitter.com/'

auth_endpoint = base_url+'oauth2/token'

auth_headers = { 'Authorization': 'Basic {}'.format(b64_encoded_key),
                'Content-Type': 'application/x-www-form-urlencoded;charset=UTF-8'}

auth_data = { 'grant_type': 'client_credentials'}

# Post OAuth2:

+ access public data 

+ Allows a registered application (*such as*) the one you created when you received your client and secret keys: to obtain a `OAuth2 Bearer Token`. 
+ This in turn will allow you to now make api calls "requests"
+ If there are too many requests in a short amount of time: you will get denied and receice an `error: 403`

https://developer.twitter.com/en/docs/basics/authentication/api-reference/token


In [15]:
# Post Request:
response = requests.post(auth_endpoint, headers=auth_headers, data=auth_data)
response.status_code

200

In [16]:
json_data = response.json()


'''
{'token_type': 'bearer',
 'access_token': 'your token will be here'}
'''

"\n{'token_type': 'bearer',\n 'access_token': 'your token will be here'}\n"

In [17]:
access_token = json_data['access_token']

# access_token

# Get Request: 

+ Notice: we are using the *OAuth 1.1* here (ability to access private data)

`parameters`: 

+ `q`: is required, it is a query based on up to 500 words using UTF-8 encoding. 

+ `count`: up to 100 tweets per page returned

+ `result_type`: 
    + `mixed`: popular and real time
    + `recent`: most recent
    + `popular`: most popular


https://developer.twitter.com/en/docs/basics/authentication/oauth-1-0a

https://developer.twitter.com/en/docs/tweets/search/api-reference/get-search-tweets

In [18]:
# Get Request:
search_headers = {'Authorization': 'Bearer {}'.format(access_token)    
}

parameters = { 'q': 'climate change',
                    'result_type': 'recent',
                'count': 50 }


search_url = base_url+'1.1/search/tweets.json'

response = requests.get(search_url, headers=search_headers, params=parameters)

+ you will see that your data is inside of `statuses`, then you have to go inside of `text` to find the actual tweet you want. There are other parameters you can extract if you need them, it just depends on what you want to evaluate later.

In [19]:
json_tweet_data = response.json()
# json_tweet_data

In [202]:
# Store Only: Tweets if you want
text=[]
for status in json_tweet_data['statuses']:
    text.append(status['text'] + '\n')
    
text[0]

"RT @heybuddy_comic: millenial: i wanna die\n\nboomer: here's climate change\n\nmillenial: not like that\n"

# Evaluating these data:

+ If we notice the dataframe datatypes which is an `object` has particular interest. Because, this is where you will have the potential of nested json objects. We can delve into this a few ways: do you want to take nested json before or after using a dataframe. 

*Either way, we will have to use some loops, recursion, if/else statements as a started!*

**Twitter Attributes**: when you parse tweets you can end up with over a 150 key values pairs for each tweet (mainly due to nesting). This can really explode depeding on how much data you are working with; so take this into consideration and read the official documentation. 

In [21]:
h=[]
for i in json_tweet_data['statuses']:
    h.append(i)

tweets_df=pd.DataFrame(h)


In [22]:
tweets_df.dtypes

created_at                    object
id                             int64
id_str                        object
text                          object
truncated                       bool
entities                      object
metadata                      object
source                        object
in_reply_to_status_id        float64
in_reply_to_status_id_str     object
in_reply_to_user_id          float64
in_reply_to_user_id_str       object
in_reply_to_screen_name       object
user                          object
geo                           object
coordinates                   object
place                         object
contributors                  object
retweeted_status              object
is_quote_status                 bool
retweet_count                  int64
favorite_count                 int64
favorited                       bool
retweeted                       bool
lang                          object
possibly_sensitive            object
extended_entities             object
d

+ I am deciding to parse from the data frame for two reasons instead directly using the data.
    + First, the data start to expand like crazy and affect my memory
    + Second, knowing what I have to deal with looking into the DF columns, makes it easier for me to work with (mentally and physically) since I have associated rows already. 

In [203]:
# nested list of [index,{}]
hh=[]
for i in range(len(json_tweet_data['statuses'])):
    for j in json_tweet_data['statuses'][i].values():
        if type(j) ==dict: 
            hh.append([i,j])

In [190]:
# from collections import defaultdict

dct_lst=defaultdict(list)

for i in hh:
    dct_lst[i[0]].append(i[1])


In [25]:
# Take our dictionary, map keys as columns, values as pd.Series and convert from 
# long to wide data with transpose (T)

DF_yay=pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in dct_lst.items() ])).T

DF_yay.head()

,0,1,2,3,4
0,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","{'id': 1686357090, 'id_str': '1686357090', 'na...",{'created_at': 'Thu Aug 13 10:14:40 +0000 2020...,NaN
1,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","{'id': 1222702755162345473, 'id_str': '1222702...",{'created_at': 'Thu Aug 13 19:54:17 +0000 2020...,NaN
2,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","{'id': 1290229633816899584, 'id_str': '1290229...",{'created_at': 'Thu Aug 13 23:06:57 +0000 2020...,NaN
3,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","{'id': 794881604, 'id_str': '794881604', 'name...",{'created_at': 'Thu Aug 13 10:14:40 +0000 2020...,NaN
4,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","{'id': 119406437, 'id_str': '119406437', 'name...",{'created_at': 'Thu Aug 13 13:41:04 +0000 2020...,NaN


In [192]:
# import json

'''
Take in the dataframe, convert to JSON then use orient to position data correctly.
json_normalize will take in the dictionary rows and convert to unnested form (unless)
its values are lists of new dictionaries

'''


json_struct=json.loads(DF_yay.to_json(orient="records")) 

In [193]:
df_exp=pd.json_normalize(json_struct)
df_exp.head()

,4,0.hashtags,0.symbols,0.user_mentions,0.urls,1.iso_language_code,1.result_type,2.id,2.id_str,2.name,2.screen_name,2.location,2.description,2.url,2.entities.description.urls,2.protected,2.followers_count,2.friends_count,2.listed_count,2.created_at,2.favourites_count,2.utc_offset,2.time_zone,2.geo_enabled,2.verified,2.statuses_count,2.lang,2.contributors_enabled,2.is_translator,2.is_translation_enabled,2.profile_background_color,2.profile_background_image_url,2.profile_background_image_url_https,2.profile_background_tile,2.profile_image_url,2.profile_image_url_https,2.profile_banner_url,2.profile_link_color,2.profile_sidebar_border_color,2.profile_sidebar_fill_color,2.profile_text_color,2.profile_use_background_image,2.has_extended_profile,2.default_profile,2.default_profile_image,2.following,2.follow_request_sent,2.notifications,2.translator_type,3.created_at,3.id,3.id_str,3.text,3.truncated,3.entities.hashtags,3.entities.symbols,3.entities.user_mentions,3.entities.urls,3.metadata.iso_language_code,3.metadata.result_type,3.source,3.in_reply_to_status_id,3.in_reply_to_status_id_str,3.in_reply_to_user_id,3.in_reply_to_user_id_str,3.in_reply_to_screen_name,3.user.id,3.user.id_str,3.user.name,3.user.screen_name,3.user.location,3.user.description,3.user.url,3.user.entities.url.urls,3.user.entities.description.urls,3.user.protected,3.user.followers_count,3.user.friends_count,3.user.listed_count,3.user.created_at,3.user.favourites_count,3.user.utc_offset,3.user.time_zone,3.user.geo_enabled,3.user.verified,3.user.statuses_count,3.user.lang,3.user.contributors_enabled,3.user.is_translator,3.user.is_translation_enabled,3.user.profile_background_color,3.user.profile_background_image_url,3.user.profile_background_image_url_https,3.user.profile_background_tile,3.user.profile_image_url,3.user.profile_image_url_https,3.user.profile_banner_url,3.user.profile_link_color,3.user.profile_sidebar_border_color,3.user.profile_sidebar_fill_color,3.user.profile_text_color,3.user.profile_use_background_image,3.user.has_extended_profile,3.user.default_profile,3.user.default_profile_image,3.user.following,3.user.follow_request_sent,3.user.notifications,3.user.translator_type,3.geo,3.coordinates,3.place,3.contributors,3.is_quote_status,3.retweet_count,3.favorite_count,3.favorited,3.retweeted,3.lang,2.entities.url.urls,3.possibly_sensitive,3,0.media,1.media,2.iso_language_code,2.result_type,3.name,3.screen_name,3.location,3.description,3.url,3.entities.url.urls,3.entities.description.urls,3.protected,3.followers_count,3.friends_count,3.listed_count,3.favourites_count,3.utc_offset,3.time_zone,3.geo_enabled,3.verified,3.statuses_count,3.contributors_enabled,3.is_translator,3.is_translation_enabled,3.profile_background_color,3.profile_background_image_url,3.profile_background_image_url_https,3.profile_background_tile,3.profile_image_url,3.profile_image_url_https,3.profile_banner_url,3.profile_link_color,3.profile_sidebar_border_color,3.profile_sidebar_fill_color,3.profile_text_color,3.profile_use_background_image,3.has_extended_profile,3.default_profile,3.default_profile_image,3.following,3.follow_request_sent,3.notifications,3.translator_type,4.created_at,4.id,4.id_str,4.text,4.truncated,4.entities.hashtags,4.entities.symbols,4.entities.user_mentions,4.entities.urls,4.entities.media,4.extended_entities.media,4.metadata.iso_language_code,4.metadata.result_type,4.source,4.in_reply_to_status_id,4.in_reply_to_status_id_str,4.in_reply_to_user_id,4.in_reply_to_user_id_str,4.in_reply_to_screen_name,4.user.id,4.user.id_str,4.user.name,4.user.screen_name,4.user.location,4.user.description,4.user.url,4.user.entities.url.urls,4.user.entities.description.urls,4.user.protected,4.user.followers_count,4.user.friends_count,4.user.listed_count,4.user.created_at,4.user.favourites_count,4.user.utc_offset,4.user.time_zone,4.user.geo_enabled,4.user.verified,4.user.statuses_count,4.user.lang,4.user.contributors_enabled,4.user.is_translator,4.user.is_translation_enabled,4.user.pr

In [ ]:
'''
df.explode() is good to take dataframe columns that have rows of lists and remove the list 
component and free up the variable according to their values for mapping. Now, if you have
lists of dictionaries it will not do anything to processing those. You have to go further.
Check documentation online.
'''

op=df_exp.explode('0.hashtags')

op.head()

In [200]:
# pd.set_option('display.max_columns', None) # allowing you to see all columns if needed

'''
 taking in thus column and converting to a pandas series  and removing lists-dictionary
using (nn).

From you have the dictionary of dictionaries to spread out (json_stru)
then (json_normalize) which will split the nested row and flatten out
'''

nn=df_exp['0.user_mentions'].apply(pd.Series)

# converting again
json_stru=json.loads(nn.to_json(orient="records")) 

# removing the json-> unnested
new_df=pd.json_normalize(json_stru)



In [ ]:
'''
Now, take the two dataframes and combine by column and 
second step: remove an columns that contain rows of all NaN values.
'''

sd_=pd.concat([df_exp, pd.json_normalize(json_stru)], axis=1)

jk=sd_.dropna(axis='columns',how='all')

In [178]:
'''
Comparing all data that contains a single column with all rows with NaN to remove.

There were ~15.6% of the 230+ columns that were useless data that I remove. 
I then compare to figure out if I did it correctly and found that I retained 84.5% data
corretly.
'''

len(sd_.columns)
len(df_exp.columns)

print(sum(sd_.isnull().values.all(axis=0))/len(sd_.columns))


print(len(jk.columns))
print(len(sd_.columns))
print(len(jk.columns)/len(sd_.columns))

0.15625
216
256
0.84375


# First, You have a few things to do to complete this:

+ Add the orignial columns of dataset that are not overlapping in data.
+ Change the column names
+ Finish unnesting the final columns.

`-------------------------`

# <font color=red>LIKE</font>, Share &

# <font color=red>SUb</font>scribe

# Citations & Help:

# ◔̯◔

https://aaronparecki.com/oauth-2-simplified/

https://medium.com/swlh/converting-nested-json-structures-to-pandas-dataframes-e8106c59976e

https://stackoverflow.com/questions/39899005/how-to-flatten-a-pandas-dataframe-with-some-columns-as-json/39906235

https://www.kaggle.com/jboysen/quick-tutorial-flatten-nested-json-in-pandas

https://github.com/amirziai/flatten

https://stackoverflow.com/questions/19736080/creating-dataframe-from-a-dictionary-where-entries-have-different-lengths

